<h1>Kapitel 10: Fehlerbehandlung (Exceptions)</h1>
<p>Grundlagen der Programmierung 1<p><h2>Holger Karl</h2>
 

<h1>Overview</h1> 

1. Überblick
1. Fehlerbehandlung durch Rückgabewerte
1. Ausnahmebehandlung (Exceptions)
1. Exceptions in Python -- Basics
1. Geschachtelte Exceptions
1. Exceptions: Typische Nutzung
1. Exceptions in Python -- Advanced Features
1. Zusammenfassung 
 


<h1>Overview</h1> 

1. **<font color="red">Überblick</font>**
1. Fehlerbehandlung durch Rückgabewerte
1. Ausnahmebehandlung (Exceptions)
1. Exceptions in Python -- Basics
1. Geschachtelte Exceptions
1. Exceptions: Typische Nutzung
1. Exceptions in Python -- Advanced Features
1. Zusammenfassung

# Setup

Main point here is to set up warnings properly for presentation and to
load tutormagic, so that we can later on use pythontutor for code
animations. 

In [ ]:
%load_ext tutormagic
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning, module='.*/IPython/.*')

# Was bisher geschah

-   Wir hatten in Funktionen, Klassen mit Sonderfällen zu tun
    -   Fehlende oder sinnlose Eingaben
    -   Unmögliche Ausführung
-   Wir haben versucht, Probleme durch Rückgabewerte dem Aufrufer
    mitzuteilen
-   Das funktioniert bestenfalls kläglich

# Dieses Kapitel

-   Ein konsistentes Vorgehen, um mit Fehler und Ausnahmen umzugehen
    -   Fehler zu signalisieren
    -   Fehler zu behandeln, weiter zu reichen oder zu ignorieren
-   Objektorientiert, also Fehler sind ebenfalls Objekte

<h1>Overview</h1> 

1. Überblick
1. **<font color="red">Fehlerbehandlung durch Rückgabewerte</font>**
1. Ausnahmebehandlung (Exceptions)
1. Exceptions in Python -- Basics
1. Geschachtelte Exceptions
1. Exceptions: Typische Nutzung
1. Exceptions in Python -- Advanced Features
1. Zusammenfassung

# Vorgehen bisher: Fehlercode

-   Kann eine Funktion nicht weiter arbeiten, gibt sie einen besonderen
    Rückgabewert zurück &#x2013; einen *Fehlercode* 
    -   Den String `Error`, die Zahl -1, &#x2026;
    -   Je nach Funktion; hoffentlich ordentlich dokumentiert
-   Der Aufrufer muss diesen Fehlercode abfragen und entsprechend
    weitermachen

# Probleme bei Fehlercodes

-   Wie Fehlercodes von normalen Rückgabewerte unterscheiden?
    -   Immer zwei Werte zurückgeben? Fehlercode und eigentliches
        Resultat?
        -   Geht nicht in allen Programmiersprachen
-   Fehlerbehandlung im Aufrufer bei aufeinander folgenden Aufrufen?
-   Was, wenn der Aufrufer nicht alle Fehler selbst behandeln kann,
    sondern an seinen Aufrufer nach oben weitergeben muss?

# Probleme bei Fehlercodes &#x2013; Beispiel

## Code ohne Fehlerbehandlung<a id="orgheadline14"></a>

In [ ]:
f()
g()
h()

## Code mit Fehlerbehandlung<a id="orgheadline15"></a>

In [ ]:
myresult = "ok"      # das gebe ich an meinen Aufrufer zurück 

result = f()
if result == "ok":
    result = g()
    if result == "ok" or result == "nicht schlimm":
        if result == "nicht schlimm":
            # Fehler reparieren
            ...
        result = h()
        if result == "ok":
            ...
        else:
            # kann ich nicht reparieren
            myresult = "h failed"
    else:
        # schlimme Fehler von g 
        ...
else:
    ... 
return myresult

# Probleme bei Fehlercodes &#x2013; Analyse

-   Man erkennt nicht mehr die Struktur
-   Das Programm ist nicht wartbar
-   Z.B. Fehlerbehandlung von `g` an verschiedenen Stellen
-   Mit dem gleichen Fehler aus verschiedenen Aufrufen kann man
    praktisch nicht umgehen

<div class="alertblock-head" style="border:3px; border-style:solid; border-color:#f96900; padding: 5px; background-color:#f96900;  line-height:1.5em; text-align:center; border-radius: 15px 15px 0px 0px; margin-bottom: 0; width: 100%;"><b>Achtung! Fehlercodes<a id="orgheadline17"></a></b></div>

Kein Wunder, dass niemand Fehlerbehandlung schreibt! 

<div class="alertblock-content" style="border:3px; border-style:solid; border-color:#f96900; padding: 5px; margin-top: 0;  line-height:1.5em; border-radius: 0px 0px 15px 15px; width: 100%; text-align:left"></div> 

<h1>Overview</h1> 

1. Überblick
1. Fehlerbehandlung durch Rückgabewerte
1. **<font color="red">Ausnahmebehandlung (Exceptions)</font>**
1. Exceptions in Python -- Basics
1. Geschachtelte Exceptions
1. Exceptions: Typische Nutzung
1. Exceptions in Python -- Advanced Features
1. Zusammenfassung

# Anforderungen Fehlerbehandlung

-   Die Darstellung des fehlerfreien Ablaufs möglichst nicht
    unterbrechen
-   Meldung des Fehlers *nicht* über Rückgabewert
-   Nicht nach jedem Aufruf einen Fehler auswerten
-   Reaktion auf Fehler an beliebiger Stelle in Aufruferkette möglich

# Mechanismus: Ausnahmen

Idee: Wir führen eine neue Art von Blocks ein 

-   Einen *geschützten Block*
    -   Fehler in diesem Block werden behandelt
-   Eine *Ausnahme* repräsentiert einen aufgetretenen Fehler
-   Solche Blöcke haben eigenen Code zur Behandlung einer Ausnahme &#x2013;
    einen *exception handler*

# Was will man über eine Ausnahme wissen?

-   Wäre es nicht nützlich zu wissen: 
    -   Welcher Fehler vorlag?
    -   Wo der Fehler auftrat?
-   Und beim Behandeln der Ausnahme dieses Wissen zu benutzen?
-   Ggf. Ähnlichkeiten zwischen Fehler auszunutzen?
    -   Eigene Fehlerkategorien zu definieren, zu verfeinern?

# Ausnahmen als Objekte

Also naheliegend: 

-   Ausnahmen als Objekte repräsentieren
-   Eine geeignete Klassenhierarchie entwerfen
-   Eigene Unterklassen erlauben
-   Beim Behandeln Unterklassen/Oberklassen unterscheiden und `is-a`
      Beziehungen ausnutzen
    -   Manchmal den speziellen Fehler, manchmal den allgemeineren Fall
        behandeln

# Ausnahmen und Ausführungsmodell

<div class="definition-head" style="border:3px; border-style:solid; border-color:#5da9e9; padding: 5px; background-color:#5da9e9;  line-height:1.5em; text-align:center; border-radius: 15px 15px 0px 0px; margin-bottom: 0; width: 100%;"><b>Definition: Ausnahme<a id="orgheadline23"></a></b></div>

Eine Ausnahme ist eine Repräsentation eines Fehlerzustands in einem
Objekt einer geeigneten Klasse. 

Eine eintretende Ausnahme *verändert das Ausführungsmodell*: Es wird
nicht die nächste Anweisung ausgeführt, sondern die Folge der
Anweisungen abgebrochen und der zugehörige Ausnahmebehandler (exception
handler) zur Ausführung gebracht. 

<div class="definition-content" style="border:3px; border-style:solid; border-color:#5da9e9; padding: 5px; margin-top: 0; border-radius: 0px 0px 15px 15px; width: 100%; text-align:left"></div>

<h1>Overview</h1> 

1. Überblick
1. Fehlerbehandlung durch Rückgabewerte
1. Ausnahmebehandlung (Exceptions)
1. Exceptions in Python -- Basics
1. Geschachtelte Exceptions
1. Exceptions: Typische Nutzung
1. Exceptions in Python -- Advanced Features
1. Zusammenfassung

# Ausnahmen: Objekte von `Exception`

-   Ausnahmen werden in Python durch Objekte der Klasse `BaseException` oder
    abgeleiteter Klassen dargestellt
-   Siehe [Klassenhierarchie der Ausnahmen](https://docs.python.org/3/library/exceptions.html#exception-hierarchy)
-   Einige Beispiele:
    -   `ArithmeticError`
        -   `ZeroDivisionError`
    -   `LookupError`
        -   `IndexError`: Index ausserhalb des gültigen Bereichs
        -   `KeyError`: Schlüssel in `dict` nicht vorhanden

# Auftreten von Exceptions

Schreiben wir ein paar fehlerhafte Programme 

## Division durch 0<a id="orgheadline26"></a>

In [ ]:
x = 5/0

## Zugriff auf nicht vorhandenen Index<a id="orgheadline27"></a>

In [ ]:
l = [4, 5, 6]
l[22]

# Auftreten von Exceptions (2)

## Nachschlagen eines nicht vorhandenen Schlüssels in `dict`<a id="orgheadline29"></a>

In [ ]:
d = {'a': 17, 'b': 42}
d['c']

# Unbehandelte Exceptions

Verhalten: 

-   Die Exception trat auf
-   Sie wurde nicht behandelt
-   Also wird das Programm mit der Fehlerausgabe abgebrochen

# Unbehandelte Exceptions an Halloween

<center>![img](./figures/foxtrot_bluescreenofdeath.png "An Halloween kann es sinnvoll sein, Exceptions nicht zu behandeln")</center>

Abbildung [61](#orgparagraph1)

# Exceptions behandeln: geschützter Block

Wir brauchen neue Schlüsselworte: 

-   `try`: Definiere einen geschützten Block
-   `except`: Nach einem geschützten Block; Code zur Fehlerbehandlung
    -   Code selbst: ein Block!
-   Assoziation von `try` und `except`: Gleiche Einrückung

In [ ]:
try:
    d = {'a': 17, 'b': 42}
    d['c']
except:
    print("Ein Fehler ist aufgetreten")

print("Es geht hier weiter")

# Semantik von `try` / `except`, Fehlerfall

-   Anweisung im Block von `try` ausführen
-   Bei Auftreten von Fehler:
    -   Mit Anweisungen nach `except` weitermachen
    -   Effekte der Anweisung des `try`-Blocks bleiben erhalten

In [ ]:
x = 17
try:
    x = 42
    y = 5/0
except:
    print("Aufgefangen")
print("x: ", x)

# Nur Fehler bestimmten Typs behandeln

`except` kann Klasse der zu behandelnden  Exception als Parameter bekommen 

In [ ]:
try:
    d = {'a': 17, 'b': 42}
    d['c']
except KeyError:
    print("Ein Fehler ist aufgetreten")

print("Es geht hier weiter")

# Nur Fehler bestimmten Typs behandeln (2)

Falls Fehler anderen Typs auftritt: Programm wird abgebrochen; keine
Ausgabe in diesem Beispiel 

In [ ]:
try:
    d = {'a': 17, 'b': 42}
    d['c']
except IndexError:
    print("Ein Fehler ist aufgetreten")

print("Es geht hier weiter")

# Auf mehrere Fehlerklassen vorbereitet sein

Auf einen `try`-Block dürfen mehrere `except`-Anweisungen folgen 

In [ ]:
try:
    d = {'a': 17, 'b': 42}
    d['c']
except KeyError:
    print("Ein Schlüssel-Fehler ist aufgetreten")
except IndexError:
    print("Ein Index-Fehler ist aufgetreten")

print("Es geht hier weiter")

# Reihenfolge mehrerer `except`

Semantik der Reihenfolge? 

-   Nimm den *ersten*  `except`-Block, der der aufgetretenen Exception entspricht

## LookupError vor KeyError<a id="orgheadline38"></a>

In [ ]:
try:
    d = {'a': 17, 'b': 42}
    d['c']
except LookupError:
    print("Ein Lookup-Fehler ist aufgetreten")
except KeyError:
    print("Ein Schlüssel-Fehler ist aufgetreten")

print("Es geht hier weiter")

# Reihenfolge mehrerer `except`

## KeyError vor LookupError<a id="orgheadline40"></a>

In [ ]:
try:
    d = {'a': 17, 'b': 42}
    d['c']
except KeyError:
    print("Ein Schlüssel-Fehler ist aufgetreten")
except LookupError:
    print("Ein Lookup-Fehler ist aufgetreten")

print("Es geht hier weiter")

# Mehrere `except`-Blöcke

Also: Die allgemeineren Exception-Klassen *nach* den spezielleren
Exception-Klassen abfangen 

<div class="alertblock-head" style="border:3px; border-style:solid; border-color:#f96900; padding: 5px; background-color:#f96900;  line-height:1.5em; text-align:center; border-radius: 15px 15px 0px 0px; margin-bottom: 0; width: 100%;"><b>Achtung! Leeres `except`<a id="orgheadline42"></a></b></div>

Vorsicht bei der Nutzung eines *leeren* `except`; damit werden *alle*
Exceptions abgefangen. Unter anderem auch der Versuch, das Programm
abzubrechen. Das ist in der Regel nicht gewünscht. 

Üblicherweise ist es besser, ein `except Exception` hier zu nutzen;
die Klasse `Exception` fasst die nicht-system-bezogenen Exceptions
zusammen ([Klassenhierarchie der Ausnahmen](https://docs.python.org/3/library/exceptions.html#exception-hierarchy)). Aber auch so ein *catch
all* ist mit Vorsicht zu benutzen (siehe später).

<div class="alertblock-content" style="border:3px; border-style:solid; border-color:#f96900; padding: 5px; margin-top: 0;  line-height:1.5em; border-radius: 0px 0px 15px 15px; width: 100%; text-align:left"></div> 

# Exception-Objekt: Aufbewahren und Auswerten

Wenn eine Exception als ein Objekt repräsentiert wird: 

-   Wie kommen wir an das Objekt?
-   Was kann uns das sagen?

## Syntax für Exception-Objekt<a id="orgheadline44"></a>

In [ ]:
try: 
    ...
except ExceptionClass as e:
    # werte Objekt e aus

# Exception-Objekt: Beispiel

In [ ]:
try:
    x = 5/0
except Exception as exc:
    print(exc)
    print(dir(exc))
    print(exc.args)

# Vorsicht: Exception-Objekt geht out of scope!

In [ ]:
try:
    x = 5/0
except Exception as exc:
    print(exc)
    print(dir(exc))
    print(exc.args)

print(exc)

## Wie aufbewahren?<a id="orgheadline47"></a>

# Fehlerfreier Fall: `else`

Was, wenn keine Exception auftrat? Code für diesen Fall? 

-   `else`-Anweisung!
-   Ähnlich wie bei Schleifen!

In [ ]:
try:
    x = 5/1
except Exception as exc:
    print(exc)
else:
    print("Alles gut")

# `else`: Semantik?

Wozu braucht man `else`? Warum nicht einfach ans Ende des  `try`
Blocks? 

## Äquivalenz?<a id="orgheadline50"></a>

In [ ]:
# Version 1:
try:
    ...
except:
    ...
else:
    Anweisungsfolge

# Version 2:
try:
    ...
    Anweisungsfolge
except:
    ...

# `else`: Typisches Muster

Häufig genutzt für eigentliche Aktion

-   Beispiel: Datei einlesen
-   (Beispiel hier noch nicht ganz verständlich; mehr später)

In [ ]:
try:
    f = open(arg, 'r')
except IOError:
    print('cannot open', arg)
else:
    print(arg, 'has', len(f.readlines()), 'lines')
    f.close()

# Und zum Abschluss eines Blocks: `finally`

Und schließlich: Code, der nach `try` immer ausgeführt werden soll 

-   Egal, ob Exception auftrat oder nicht
-   `finally`-Abschnitt: Wird stets nach `except` oder `else` aufgeführt
-   Zweck: Aufräumen, z.B. Datei schließen

In [ ]:
try:
    ...
except:
    ...
else:
    ...
finally:
    ...

# `finally` und Exceptions

Trat eine Exception auf, die von keinem `except` behandelt wurde, so
wird `finally` ausgeführt und die Exception tritt danach wieder auf! 

-   Beispiel:

In [ ]:
def divide(x, y):
    try:
        result = x / y
    except ZeroDivisionError:
        print("division by zero!")
    else:
        print("result is", result)
    finally:
        print("executing finally clause")

divide(2, 1)
divide(2, 0)
# divide("2", "1")

# Exceptions auslösen

Manchmal sinnvoll: Exceptions bewusst auslösen 

-   Für Tests, gezieltes Auslösen von Fehlerbehandlung woanders
-   Schlüsselwort `raise` mit Argument Exceptionklasse 
    -   Konstruktor-Parameter werden aufbewahrt und durch `print`
            ausgegeben
    -   Nützlich bei Fehlersuche!

In [ ]:
raise NameError('HiThere')

# Exceptions in Funktionsaufrufen

Was passiert, wenn 

-   in Funktion eine Exception aufritt,
-   Aufruf in `try`-Bock war?

In [ ]:
def f(nenner):
    x = 5/nenner
    print("nach Division")
    return x

try:
    x = f(0)
except ArithmeticError:
    print("Hoppala")
    x = 42

# `try` &#x2013; Definition

<div class="definition-head" style="border:3px; border-style:solid; border-color:#5da9e9; padding: 5px; background-color:#5da9e9;  line-height:1.5em; text-align:center; border-radius: 15px 15px 0px 0px; margin-bottom: 0; width: 100%;"><b>Definition: `try`<a id="orgheadline57"></a></b></div>

Eine `try`-Anweisung besteht aus 

-   dem ~try-Block an sich
-   keinem, einem oder mehreren `except`
-   einem optionalen `else`-Teil
-   einem optionalen `finally`-Teil

Wenn ein `else` auftaucht, muss mindestens ein `except` vorhanden
sein.  Die Reihenfolge ist wie angeben.

Tritt eine Exception in einem Funktionsaufruf auf, so wird die
Aufruffolge so lange abgebrochen, bis ein `try`-Block mit passendem
`except` gefunden wird. Wird keiner gefunden, wird das Programm
abgebrochen. 

<div class="definition-content" style="border:3px; border-style:solid; border-color:#5da9e9; padding: 5px; margin-top: 0; border-radius: 0px 0px 15px 15px; width: 100%; text-align:left"></div>

<h1>Overview</h1> 

1. Überblick
1. Fehlerbehandlung durch Rückgabewerte
1. Ausnahmebehandlung (Exceptions)
1. Exceptions in Python -- Basics
1. **<font color="red">Geschachtelte Exceptions</font>**
1. Exceptions: Typische Nutzung
1. Exceptions in Python -- Advanced Features
1. Zusammenfassung

# `try` in `try` ?

-   Ein `try`-Block hat Anweisungen
-   `try` selbst ist eine Anweisung
-   Also verschachteln möglich?

In [ ]:
try:
    ...
    try:
        ...
    except ExceptionClass2:
        ...
    ...
except ExceptionClass1:
    ...

# `try` in `try` !

-   Warum auch nicht!
-   Semantik ist naheliegend:
    -   Exceptions im inneren `try` werden zunächst durch dessen
        `except`-Anweisungen behandelt
    -   Falls dort nicht behandelt, eskaliert die Exception zur äußeren
        `except`-Anweisung

# `try` in `try` : Beispiel

In [ ]:
try:
    # nenner = 0 
    nenner = 10**-10000
    try:
        x = 5/nenner
    except OverflowError:
        print("das wird zu groß")
        x = 0
except ArithmeticError as e:
    print("irgendwas arithmatisches ging schief: ", e)
    x = 42

print(x)

# `try`, Funktionsaufruf, `try`

-   Typisch: 
    -   In `try`-Block Funktion aufrufen
    -   Diese Funktion hat selbst einen `try`-Block
-   Spannender Punkt:  In Funktion nicht behandelte Exception
-   Erwartungshaltung: Äußerer `try`-Block zum Zug?
    -   Funktionsausführung abgebrochen

# `try`, Funktionsaufruf, `try`  : Beispiel

In [ ]:
def f(nenner):
    try:
        x = 5/nenner
    except OverflowError:
        print("das wird zu groß")
        x = 0
    return x

try:
    # nenner = 0 
    nenner = 10**-10000
    val = f(nenner)
except ArithmeticError as e:
    print("irgendwas arithmatisches ging schief: ", e)
    val = 42

print(val)

# Ein `except`, alle `finally`

-   Eine Exception wird nur durch *ein* ~except behandelt
-   Aber es werden *alle* umschließenden *finally*-Klauseln ausgeführt

## Beispiel<a id="orgheadline64"></a>

In [ ]:
try:
    # irgendwas
    try:
        try:
            raise Exception("meine!")
        except:
            print("im dritten except")
        finally:
            print("im dritten finally")
    except:
        print("im zweiten except")
    finally:
        print("im zweiten finally")
except:
    print("im ersten except")
finally:
    print("im ersten finally")

# Ein `except`, alle `finally` &#x2013; Beispiel 2

Egal, ob direkt geschachtelt oder in Funktionen &#x2013; gleiches Verhalten 

In [ ]:
def action2():
    print(1 + [])            # Generate TypeError

def action1():
    try:
        action2()
    except TypeError:        # Most recent matching try
        print('inner except')
    finally:
        print('inner finally')

try:
    action1()
except TypeError:            # Here, only if action1 re-raises
    print('outer except')
finally:
    print('outer finally')

# Geschachtelte `try`: Realisierung

-   Intern werden `try`-Kontexte auf einem Stack organisiert
    -   Ähnlich zu Unterprogrammaufrufen
    -   In einem Kontext sind u.a. die `except`-Stellen abgelegt &#x2013; welche
        Klassen, an welcher Stelle wird fortgesetzt?
-   Bei Auftreten einer Exception wird dieser Stack durchsucht

<h1>Overview</h1> 

1. Überblick
1. Fehlerbehandlung durch Rückgabewerte
1. Ausnahmebehandlung (Exceptions)
1. Exceptions in Python -- Basics
1. Geschachtelte Exceptions
1. **<font color="red">Exceptions: Typische Nutzung</font>**
1. Exceptions in Python -- Advanced Features
1. Zusammenfassung

# Typische Muster, wie Exceptions genutzt werden

-   Bisher: Mechanik der Exceptions, mit ein paar Beispielen
-   Schauen wir uns typische Muster an, wie Exceptions häufig eingesetzt
    werden
    -   Fehlerfälle behandeln
    -   Programmfluss steuern
-   Benutzen Sie das mit Vorsicht; nicht alles hier ist immer sinnvoll
    -   Aber Sie müssen solche Muster zumindest wiedererkennen können

# Exceptions für Fehlerfälle im eigentlichen Sinne

Ziel kann nicht erreicht werden, wir brauchen Sonderbehandlung: 

-   Arithmetische Fehler: Division durch Null, &#x2026;
-   Ein-/Ausgabe: Datei nicht vorhanden, keine Berechtigung, &#x2026;
    -   Unterschiedliche Fehler können gleich behandelt werden!

# Exceptions anstelle besonderer  Rückgabewerte

-   Funktion, die
    -   bestimmtes Ereignis kommunizieren muss
    -   aber keinen besonderen Wert (*sentinel value*)  verwenden kann, weil alle Werte
        vorkommen könnten
-   Beispiel: Suchfunktion

In [ ]:
class Found(Exception): pass

def searcher():
    if ...success...:
        raise Found()            # Raise exceptions instead of returning flags
    else:
        return

try:
    searcher()
except Found:                    # Exception if item was found
    ...success...
else:                            # else returned: not found
    ...failure...

# Mehrere geschachtelte Schleifen abbrechen

-   `break` bricht nur die innerste Schleife ab
-   Mit Exceptions:

In [ ]:
class Exitloop(Exception): pass

try:
    while True:
        while True:
            for i in range(10):
                # break would just not do the job 
                if i > 3: raise Exitloop          
                print('loop3: %s' % i)
            print('loop2')
        print('loop1')
except Exitloop:
    print('continuing')

Anmerkung: Zeigen Sie das keinem Software-Techniker :-) 

# Exceptions als Anzeichen für erwartetes Verhalten

-   Typisch: Mit Exception auf das Ende einer Eingabe reagieren

In [ ]:
while True:
    try:
        # Eingabe einlesen (z.B. von Tastatur, Datei)
        line = input()           
    except EOFError:
        # EOFError: keine Eingabe mehr vorhanden 
        break                    
    else:
        # hier nächste Zeile verarbeiten
        ...

# Was gehört in `try`?

Zusammenfassend: Was sollte in einem `try`-Block ausgeführt werden? 

-   Anweisungen, bei denen häufig Fehler auftreten: Interaktion mit
    Nutzer oder Betriebssystem
-   Lange Funktionen: Viele `try` in Funktion, oder ein `try` beim
    Aufruf?
    -   Viele `try`: Exception an der richtigen Stelle; Code wird
        unleserlich
    -   Ein `try` für Aufruf: Gefahr den zu vergessen!
        Ggf. Wrapper-Funktion (insbes. sinnvoll in Klasse mit eigentlicher
        Funktion als Unterstrich-Version)

# Was abfangen?

-   Nicht zu viel: Klasse nicht zu allgemein 
    -   Ein bloßes `except:` ohne Klasse ist gefährlich; behandelt auch
        Fehler, die eigentlich nicht behandelt werden sollen an dieser
        Stelle
        -   Z.B. Programmabbruch
        -   Fehler die weiter außen sinnvoller behandelt werden können
    -   In der Regel sollte mindestens auf `except Exception` eingeschränkt
        werden, um Programmablauf-Exceptions nicht zu behandeln
-   Nicht zu wenig: Klasse nicht zu speziell
    -   Insbes. wenn eigene Exceptions als normal behandelt werden sollen
    -   Was passiert, wenn man weitere normale Exceptions hinzufügt?
    -   Über Klassenhierarchie der eigenen Exceptions nachdenken!

# Coding Style: EAFP vs. LBYL

Sprachen mit Exceptions (nicht nur Python) führen oft zu einem Coding-Style, der diese
umfassend benutzt: 

-   Der Normalfall wird in einem `try` aufgeschrieben
-   Dann kommen ggf. viele `except` mit Reparatur-Code
-   Zusammengefasst: *Easier to ask forgiveness than permission* (EAFP)

Alternative in Sprachen ohne Exceptions: Vor einer Aktion prüfen, ob
diese durchgeführt werden kann 

-   Durch viele, viele `if`-Anweisungen charakterisiert
-   Zusammengefasst: *Look before you leap* (LBYL)

<h1>Overview</h1> 

1. Überblick
1. Fehlerbehandlung durch Rückgabewerte
1. Ausnahmebehandlung (Exceptions)
1. Exceptions in Python -- Basics
1. Geschachtelte Exceptions
1. Exceptions: Typische Nutzung
1. Exceptions in Python -- Advanced Features
1. Zusammenfassung

# Eigene Exceptions

-   Fehlersuche in einem komplexen Programm wird kompliziert
-   Wir würden gerne mehr Information aufbewahren und ausgeben
-   Und mehr Klassen von Exceptions unterscheiden können, zur genaueren
    Fehlerbehandlung

Lösung: Exceptions sind Objekt; Klassenhierarchie erweitern! 

# Eigene Exception-Klassen

Beispiel: Detaillierte Ausgabe von Parametern 

In [ ]:
class MyExcp(Exception):
    def __str__(self):
        return ("MyExcept deals with this: {}, {}. {}".format(
            self.args[0], self.args[1], self.args[2])
            )

try:
    raise MyExcp("a", 2, 3)
except MyExcp as e:
    print(e)

# Eigene Exception-Klassen (2)

Beispiel: Exceptions in Datei mitprotokollieren 

In [ ]:
class MyExcp(Exception):
    logfile = "mylogs.txt"
    def logerror(self):
        log = open(self.logfile, 'a')
        print("MyExcept deals with this: {}, {}. {}".format(
            self.args[0], self.args[1], self.args[2]),
            file=log)
        log.close()

try:
    raise MyExcp("a", 2, 3)
except MyExcp as e:
    # let's log this:
    e.logerror()

# Exceptions in `except`?

Was passiert, wenn der Code in `except` selbst wieder einen Fehler
produziert? 

-   Option 1: Weitere `except`-Klauseln des gleichen `try` werden
    benutzt
    -   Unübersichtlich&#x2026;
-   Option 2: `finally` ausführen, und danach die Exception als neu
    aufgetreten ansehen
    -   Und natürlich den `except`-Block nach Fehler abbrechen

# Exceptions in `except`? &#x2013; Beispiel

In [ ]:
try:
    try:
        x = 5/0
    except ArithmeticError:
        print("Anfang ArithmeticError Handler")
        raise IndexError
        print("Ende ArithmeticError Handler")
    except IndexError:
        print("im inneren IndexError Handler")
    finally:
        print("Finally im inneren try")
except:
    print("Im outer exception handler")

# Exceptions in `except`  &#x2013; Vorsicht

<center>![img](./figures/exception-in-except.png "Exceptions beim Behandeln von Exceptions")</center>

# `sys.exc_info` und Tracebacks

Es gibt mehr über eine Exception zu erfahren! 

-   Information über gerade behandelte  Exception: eingebaute Funktion `sys.ecx_info`
    -   Liefert 3-Tuple (type, value, traceback):
        -   Type: Die Klasse der aufgetretenen Exception
        -   Value: Das Objekt dieser Klasse
        -   Traceback: siehe unten
    -   Nur sinnvoll in except-Klausel, sonst (None, None, None)

# `sys.exc_info`: Beispiel

In [ ]:
# import machen wir in folgendem Kapitel 
import sys
try:
    raise IndexError("ein absichtlicher Fehler")
except Exception:
    print(sys.exc_info())

## Beobachtung<a id="orgheadline83"></a>

Obwohl wir nur nach `Exception` fragen, bekommen wir über
`sys.exc_info()` genauere Informationen über die tatsächliche Klasse 

# Traceback

-   Traceback: Folge der Aufrufe, die zu der Exception geführt haben
    -   Welche Funktionsaufrufe?
    -   Welche Zeilen?
-   Haben wir schon in den Beispielen oben gesehen!
-   Steht als eigenes Objekt in `sys.exc_info` zur Verfügung
-   Zur Verarbeitung braucht man Hilfsfunktionen

# Traceback &#x2013; Beispiel

In [ ]:
import sys
import traceback 
def f():
    print("Anfang f")
    try:
        g()
    except IndexError:
        print("Index Error")
    print("Ende f")

def g():
    raise ArithmeticError 

try:
    f()
except Exception:
    exc = sys.exc_info()
    tb = traceback.extract_tb(exc[2])
    print(tb)

# Aufräumarbeiten: Context managers

Häufiges Muster bei `finally`: Aufräumarbeiten durchführen 

-   Datei schließen und ähnliches
-   Auch oft: Aufräumarbeiten hängen von einem bestimmten Objekt ab, das
    in `try` benutzt wird
    -   Und sind immer die gleichen, egal was sonst passiert

Also Muster: 

In [ ]:
try:
    # erzeuge ein Objekt 
    obj = ...
    # Verarbeite
    ...
finally:
    obj.aufraeum()

# Aufräumarbeiten: Context managers

-   Idee: Wir legen fest, wie man bei einem Objekt (genauer: bei der
    Klasse) die Aufräumarbeiten identifizieren kann
    -   Z.B.: eine Methode `__exit__`
-   Dann: Kompakte Version möglich durch neues Schlüsselwort: `with ... as`
    -   Wichtige Fälle verdienen explizite Unterstützung durch Sprache!
-   [Details](http://proquest.tech.safaribooksonline.de/book/programming/python/9781449355722/34dot-exception-coding-details/ch34s06_html) sind umfangreicher

# `with ... as` als Ersatz für `try`-Muster

## Kompakt mit `with`<a id="orgheadline89"></a>

In [ ]:
with Ausdruck as variable:
    ...

## Entspricht `try`<a id="orgheadline90"></a>

In [ ]:
try:
    variable = Ausdruck
    ...
finally:
    variable.__exit__()

<h1>Overview</h1> 

1. Überblick
1. Fehlerbehandlung durch Rückgabewerte
1. Ausnahmebehandlung (Exceptions)
1. Exceptions in Python -- Basics
1. Geschachtelte Exceptions
1. Exceptions: Typische Nutzung
1. Exceptions in Python -- Advanced Features
1. **<font color="red">Zusammenfassung</font>**

# Zusammenfassung

-   Exceptions sind ein Mittel, den Programmablauf in Fehlerfällen und
    Sonderfällen gezielt zu beeinflussen
-   Exceptions können an einer Stelle eines Programms auftreten und an
    anderen Stellen behandelt werden
    -   Insbes. bei geschachtelten Funktionsaufrufen
-   Exceptions werden häufig in bestimmten Mustern benutzt (Lesbarkeit!)

  